### 1.Import Libary

In [31]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer
import numpy as np
import collections
from tqdm.auto import tqdm
import evaluate
import wandb
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### 2.Data Preprocessing

In [7]:


def read_dataset(file_path):
    """Đọc dataset từ file CSV."""
    df = pd.read_csv(file_path)
    df['context'] = df['context'].astype(str)
    df['question'] = df['question'].astype(str)
    df['answer'] = df['answer'].astype(str)
    return df

def find_start_index(context, answer):
    """Tìm chỉ số bắt đầu của answer trong context."""
    return str(context).find(str(answer))

def prepare_dataset(df):
    TRAIN_RATIO = 0.8
    VAL_RATIO = 0.1
    """Chuẩn bị dataset cho huấn luyện, xác thực và kiểm tra."""
    df['start_index'] = df.apply(lambda row: find_start_index(context=row['context'], answer=row['answer']), axis=1)
    df = df[df['start_index'] != -1]

    dataset_temp = []
    for _, row in df.iterrows():
        sample = {
            'context': row['context'],
            'question': row['question'],
            'answer': {'text': [row['answer']], 'answer_start': [row['start_index']]}
        }
        dataset_temp.append(sample)

    dataset = pd.DataFrame(dataset_temp)

    num_of_total_sample = len(dataset)
    num_of_train_sample = TRAIN_RATIO * num_of_total_sample
    num_of_val_sample = VAL_RATIO * num_of_total_sample

    train_set = dataset.sample(n=int(num_of_train_sample), random_state=42)
    dataset.drop(index=train_set.index, inplace=True)

    val_set = dataset.sample(n=int(num_of_val_sample), random_state=42)
    dataset.drop(index=val_set.index, inplace=True)

    return Dataset.from_pandas(train_set), Dataset.from_pandas(val_set), Dataset.from_pandas(dataset)

def preprocess_training_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ huấn luyện cho mô hình."""
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_test_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ cho bộ kiểm tra và xác thực."""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["question"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


### 3.Fine-Tuning Model

> Function Evaluation

In [3]:

metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples, n_best, max_answer_length, metric):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["question"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["question"], "answers": ex["answer"]} for ex in examples]
    
    for i, ii in zip(predicted_answers, theoretical_answers):
        print("-"*99)
        print(f"Pred: {i['prediction_text']}")
#         print("*"*20)
        print(f"Goal: {ii['answers']['text'][0]}")
    
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

> Function Load Model and Load tokenizer

In [9]:
def load_model_and_tokenizer(model_checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    return model, tokenizer

> declare hyperparameters

In [5]:
wandb.login(key = '8a5cbfdaa29778a896996cc679358b1d96cf66b0')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hdang1696 (hdang1696-no-work-experience). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hdang\_netrc


True

In [11]:

MODEL_CHECKPOINT = "google-bert/bert-base-multilingual-cased"
MAX_LENGTH = 256
STRIDE = 50
N_BEST = 50
MAX_ANSWER_LENGTH = 512
TRAINING_ARGS = TrainingArguments(
    output_dir='../output',
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="no",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=100,
    weight_decay=0.01,
    fp16=True,
)


c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


> Call functions read dataset

In [15]:
df = read_dataset('../data/data_origin/data_train/ALQAC.csv')

In [8]:
df.head(5)

,context,question,answer
0,Tội chiếm đoạt hoặc hủy hoại di vật của tử sỹ\...,Chiếm đoạt di vật của tử sĩ có thể bị phạt tù ...,07 năm
1,Tù chung thân\n\nTù chung thân là hình phạt tù...,"Theo Bộ luật Hình sự năm 2017, không áp dụng h...",18 tuổi
2,Tuổi chịu trách nhiệm hình sự\n\n1. Người từ đ...,"Theo Bộ luật Hình sự năm 2017, người từ đủ bao...",16 tuổi
3,Cảnh cáo\n\nCảnh cáo được áp dụng đối với ngườ...,Người phạm tội ít nghiêm trọng và có nhiều tìn...,Cảnh cáo
4,Tử hình\n\n1. Tử hình là hình phạt đặc biệt ch...,Trong trường hợp người bị kết án tử hình được ...,tù chung thân


> call function  prepare_dateset

In [16]:
train_set, val_set, test_set = prepare_dataset(df)

> call function Load model and tokenizer

In [17]:
model, tokenizer = load_model_and_tokenizer(MODEL_CHECKPOINT)

c:\Users\hdang\.virtualenvs\machinelearning-lTTH8rYd\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> Embedding 

In [18]:

train_dataset = train_set.map(
        lambda examples: preprocess_training_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
val_dataset = val_set.map(
        lambda examples: preprocess_validation_test_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
test_dataset = test_set.map(
        lambda examples: preprocess_validation_test_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
    

Map:   0%|          | 0/415 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

> training process

In [12]:
trainer = Trainer(
        model=model,
        args=TRAINING_ARGS,
        train_dataset=train_dataset,
        tokenizer=tokenizer,
    )
    
trainer.train()
    
trainer.save_model(r'../output')


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/4500 [00:00<?, ?it/s]

KeyboardInterrupt: 

> Evaluation

In [ ]:
predictions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predictions    
results = compute_metrics(start_logits, end_logits, test_dataset, test_set, N_BEST, MAX_ANSWER_LENGTH)
print(results)

In [32]:
df_train = pd.read_csv('../data/data_origin/data_train/train.csv')
df_train.head(5)

,question,context,answer
0,"Nghỉ việc, có phải trả thẻ bảo hiểm y tế?","Do nhu cầu chuyển đổi công việc nên đầu năm 2016 tôi đơn phương chấm dứt hợp đồng lao động. Khi tôi nghỉ thì cơ quan yêu cầu tôi trả lại thẻ bảo hiểm y tế. Việc này có đúng pháp luật không? Các loại bảo hiểm khác tôi sẽ tự đóng thì các chế độ sau này có bị ảnh hưởng gì không?\nVi Mạnh Hà (tỉnh Khánh Hòa)\nLuật sư LÊ VĂN HOAN, Trưởng Văn phòng Luật sư Lê Văn (TP.HCM), trả lời: Theo Điều 15 Luật Bảo hiểm y tế năm 2008 (sửa đổi, bổ sung năm 2014) quy định hằng tháng người sử dụng lao động đóng bảo hiểm y tế cho người lao động và trích tiền đóng bảo hiểm y tế từ tiền lương của người lao động để nộp cùng một lúc vào quỹ bảo hiểm y tế. Do đó khi người lao động nghỉ việc thì người sử dụng lao động phải có trách nhiệm báo tăng, giảm với cơ quan bảo hiểm xã hội và thu hồi trả lại thẻ bảo hiểm y tế (nếu thẻ đó còn hiệu lực). Đồng thời doanh nghiệp cũng chốt sổ bảo hiểm xã hội cho người lao động.\nNhư vậy, việc doanh nghiệp yêu cầu bạn trả lại thẻ bảo hiểm y tế là đúng luật. Bạn có quyền tham gia bảo hiểm y tế tự nguyện cũng như bạn có quyền tham gia bảo hiểm xã hội tự nguyện (nếu không thuộc đối tượng tham gia bảo hiểm xã hội bắt buộc quy định tại khoản 1 Điều 2 Luật Bảo hiểm xã hội).\nTuy nhiên, người tham gia bảo hiểm xã hội tự nguyện chỉ được hưởng chế độ hưu trí và tử tuất. Trong khi người tham gia bảo hiểm xã hội bắt buộc được hưởng các chế độ gồm ốm đau; thai sản; tai nạn lao động, bệnh nghề nghiệp; hưu trí; tử tuất.","Đúng theo quy định của Luật Bảo hiểm y tế, khi bạn nghỉ việc, doanh nghiệp có quyền yêu cầu bạn trả lại thẻ bảo hiểm y tế (nếu thẻ đó còn hiệu lực). Bạn cũng có quyền tham gia bảo hiểm y tế tự nguyện và bảo hiểm xã hội tự nguyện (nếu không thuộc đối tượng tham gia bảo hiểm xã hội bắt buộc). Tuy nhiên, chỉ có người tham gia bảo hiểm xã hội bắt buộc mới được hưởng các chế độ bảo hiểm như ốm đau, thai sản, tai nạn lao động, bệnh nghề nghiệp, hưu trí và tử tuất. Người tham gia bảo hiểm xã hội tự nguyện chỉ được hưởng chế độ hưu trí và tử tuất."
1,"Chị chồng muốn hưởng tài sản thừa kế của chồng, được không?","Vợ chồng tôi có tài sản chung là căn nhà trị giá năm tỉ đồng ở TP.HCM. Năm 2018, chồng tôi mất không để lại di chúc, tôi vẫn tiếp tục sử dụng căn nhà này. Năm 2021, cha mẹ chồng tôi mất cũng không để lại di chúc. \nMới đây, chị chồng tôi yêu cầu bán căn nhà chung của hai vợ chồng tôi để chia cho chị ấy.\n Cho tôi hỏi, theo quy định pháp luật thì tôi có phải chia tài sản là căn nhà của vợ chồng tôi cho người chị chồng không?\nBạn đọc có địa chỉ mail nguyenhanh….@gmail.com\nNgười dân đến văn phòng công chứng thực hiện các thủ tục công chứng. Ảnh: NGUYỄN HIỀN\nLuật sư Hoàng Anh Sơn, Đoàn Luật sư TP.HCM, trả lời: Vì đây là tài sản chung của hai vợ chồng và khi chồng bạn mất không để lại di chúc nên căn cứ quy định tại điểm a, khoản 1 Điều 650 Bộ luật Dân sự thì giá trị một nửa căn nhà là di sản của chồng bạn sẽ được chia thừa kế theo pháp luật.\nTại Điều 651 Bộ luật Dân sự 2015, những người thừa kế theo pháp luật thứ tự như sau: \nHàng thừa kế thứ nhất gồm: Vợ, chồng, cha đẻ, mẹ đẻ, cha nuôi, mẹ nuôi, con đẻ, con nuôi của người chết. \nNhững người thừa kế cùng hàng được hưởng phần di sản bằng nhau.\nNhư vậy, phần di sản là một nửa căn nhà của chồng bạn khi mất sẽ được chia đều cho bạn, cha, mẹ của chồng bạn.\n Tuy nhiên, năm 2021 cha và mẹ chồng đều mất (mất sau chồng bạn) nên phần di sản mà cha, mẹ chồng được hưởng từ chồng bạn sẽ được chia thừa kế theo pháp luật. Theo hàng thừa kế thứ nhất của cha, mẹ chồng là chị chồng bạn. \nVì thế, chị chồng của bạn sẽ được hưởng phần di sản thừa kế mà cha mẹ chồng được hưởng từ chồng bạn.","Theo quy định pháp luật, chị chồng của bạn sẽ được hưởng phần di sản thừa kế mà cha mẹ chồng được hưởng từ chồng bạn."
2,"Bị sàm sỡ ở trạm xe buýt, phải làm sao?","Tôi thường xuyên đi làm bằng xe buýt. Tôi thường thấy một người đàn ông hay lảng vảng ở khu vực nhà chờ xe buýt gần công ty. Anh ta vạch “của quý” ra và buông lờ

In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch
model_path = './model_2/'  # Đường dẫn tới mô hình đã được lưu
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)



In [6]:
def predict_answer(context, question, model, tokenizer, max_length=512, stride=200):
    # Tokenize câu hỏi và context
    inputs = tokenizer(
        question,
        context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_tensors="pt",
        padding="max_length",
    )

    # Dự đoán bằng mô hình đã fine-tuned
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy giá trị logits cho start và end position
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Tìm các index tốt nhất cho start và end position
    start_index = torch.argmax(start_logits, dim=-1).item()
    end_index = torch.argmax(end_logits, dim=-1).item()

    # Giải mã câu trả lời từ context
    answer = tokenizer.decode(inputs['input_ids'][0][start_index:end_index+1], skip_special_tokens=True)
    
    return answer

In [9]:
context = """
Điều 53. Những nội dung cán bộ, công chức, viên chức, người lao động tham gia ý kiến trước khi người đứng đầu cơ quan, đơn vị quyết định\n1. Giải pháp thực hiện chủ trương, đường lối của Đảng, chính sách, pháp luật của Nhà nước liên quan đến chức năng, nhiệm vụ của cơ quan, đơn vị.\n2. Kế hoạch công tác hằng năm của cơ quan, đơn vị.\n3. Tổ chức phong trào thi đua của cơ quan, đơn vị.\n4. Báo cáo sơ kết, tổng kết của cơ quan, đơn vị.\n5. Các biện pháp cải tiến tổ chức, hoạt động và lề lối làm việc; phòng, chống tham nhũng, tiêu cực; thực hành tiết kiệm, chống lãng phí, chống quan liêu, phiền hà, sách nhiễu Nhân dân.\n6. Kế hoạch tuyển dụng, đào tạo, bồi dưỡng cán bộ, công chức, viên chức; bầu cử, bổ nhiệm cán bộ, công chức, viên chức.\n7. Thực hiện chế độ, chính sách liên quan đến quyền và lợi ích của cán bộ, công chức, viên chức, người lao động.
"""
question = "Hãy cho tôi biết Giải pháp thực hiện chủ trương, đường lối của Đảng "

# Dự đoán kết quả
predicted_answer = predict_answer(context, question, model, tokenizer)
print(f"Question: {question}")
print(f"Predicted Answer: {predicted_answer}")

Question: Hãy cho tôi biết Giải pháp thực hiện chủ trương, đường lối của Đảng 
Predicted Answer: Giải pháp thực hiện chủ trương, đường lối của Đảng, chính sách, pháp luật của Nhà nước liên quan đến chức năng, nhiệm vụ của cơ quan, đơn vị.
